# Math 377 Fall 2018

#### Name:
#### Section:

**Documentation Statement:**

## Project: Data Collection, Summarization, Inference and Prediction
## Building a Naive Bayes Classifier

This project is designed to cover many of the main ideas of the entire course. We will go into a little more detail with the data collection and prediction. Ultimately we want to predict if an email is spam. In the course of doing this, we will collect data, clean it up, work with string data, make a simple inference, and then build a naive bayes model from the ground up. We will then bring in the package `scikit-learn` to check our work and add extra functionality.

By the end of project, you should know how to:

1. Find and import data.
2. Use regular expressions to edit string data.
3. Determine if a word helps to identify an email as spam or not.
4. Create a function to predict the type of email using the ideas of Bayesian Classification.
5. Assess your model and propose improvements.

**Advice.** Develop your answers incrementally. To perform a complicated table manipulation, break it up into steps, perform each step on a different line, give a new name to each result, and check that each intermediate result is what you expect by displaying it. You can add additional names or functions to the provided cells in order to organize your work. 

**Authorized Resources:** Anyone and anything.

### 1. Background Information 

There are a couple of reference papers that may be of interest to explore. The first is "Better Bayesian Filtering" by Paul Graham,http://bit.ly/1ycPbiy. The second is "A Plan for Spam" also by Paul Graham, http://bit.ly/1ycPcmA

### 2. Load Packages  

To get started, load `datascience`, `numpy`, `plots`, and `pandas`.

In [3]:
import datascience
import numpy as np
import pandas as pd
#import scikit-learn
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

### 3. Get Data

We are going to use data from the [Apache SpamAssasian](https://spamassassin.apache.org/) website. In particular we want data from their public corpus, see the readme document at https://spamassassin.apache.org/old/publiccorpus/.

We have provided you the data in a zip file, spam.zip, because the data on the website is in a `tar` and `bz2` zip format. There are functions in python that allow you to address these types of file, see https://docs.python.org/3/library/tarfile.html, but it is not worth the extra coding effort at this point.

You should extract the zip file and place the three folders in a folder called data under the directory where you are running this notebook. The three folders are  
easy_ham  
spam  
hard_ham  

The files names are the message number and the MD5 checksum. This is not convenient nomenclature. To access the files we will use the `glob` package, https://docs.python.org/3/library/glob.html. We also use the `os` package to get the current working directory on your machine. Below is the code to open one file to view its contents. 

Note: using the `with` argument will automatically close the file upon completion of the loop.

In [4]:
import glob,re,os

In [5]:
# Get working directory, assumed to be
cwd = os.getcwd()
path=cwd +'\\data\\*\\*'
path

'C:\\Users\\Brad.Warner\\Documents\\Classes\\Math 377\\Fall 2018\\data\\*\\*'

In [6]:
glob.glob(path)[:10] #Returns list with all the files in it, we will look at first 10.

['C:\\Users\\Brad.Warner\\Documents\\Classes\\Math 377\\Fall 2018\\data\\easy_ham\\0001.ea7e79d3153e7469e7a9c3e0af6a357e',
 'C:\\Users\\Brad.Warner\\Documents\\Classes\\Math 377\\Fall 2018\\data\\easy_ham\\0002.b3120c4bcbf3101e661161ee7efcb8bf',
 'C:\\Users\\Brad.Warner\\Documents\\Classes\\Math 377\\Fall 2018\\data\\easy_ham\\0003.acfc5ad94bbd27118a0d8685d18c89dd',
 'C:\\Users\\Brad.Warner\\Documents\\Classes\\Math 377\\Fall 2018\\data\\easy_ham\\0004.e8d5727378ddde5c3be181df593f1712',
 'C:\\Users\\Brad.Warner\\Documents\\Classes\\Math 377\\Fall 2018\\data\\easy_ham\\0005.8c3b9e9c0f3f183ddaf7592a11b99957',
 'C:\\Users\\Brad.Warner\\Documents\\Classes\\Math 377\\Fall 2018\\data\\easy_ham\\0006.ee8b0dba12856155222be180ba122058',
 'C:\\Users\\Brad.Warner\\Documents\\Classes\\Math 377\\Fall 2018\\data\\easy_ham\\0007.c75188382f64b090022fa3b095b020b0',
 'C:\\Users\\Brad.Warner\\Documents\\Classes\\Math 377\\Fall 2018\\data\\easy_ham\\0008.20bc0b4ba2d99aae1c7098069f611a9b',
 'C:\\Users\\Bra

Let's read in the first file and print out the contents.

In [11]:
with open(glob.glob(path)[0],'r') as file: #the 'r' tells python to read the file only.
    for line in file:
        print(line)

From exmh-workers-admin@redhat.com  Thu Aug 22 12:36:23 2002

Return-Path: <exmh-workers-admin@example.com>

Delivered-To: zzzz@localhost.netnoteinc.com

Received: from localhost (localhost [127.0.0.1])

	by phobos.labs.netnoteinc.com (Postfix) with ESMTP id D03E543C36

	for <zzzz@localhost>; Thu, 22 Aug 2002 07:36:16 -0400 (EDT)

Received: from phobos [127.0.0.1]

	by localhost with IMAP (fetchmail-5.9.0)

	for zzzz@localhost (single-drop); Thu, 22 Aug 2002 12:36:16 +0100 (IST)

Received: from listman.example.com (listman.example.com [66.187.233.211]) by

    dogma.slashnull.org (8.11.6/8.11.6) with ESMTP id g7MBYrZ04811 for

    <zzzz-exmh@example.com>; Thu, 22 Aug 2002 12:34:53 +0100

Received: from listman.example.com (localhost.localdomain [127.0.0.1]) by

    listman.redhat.com (Postfix) with ESMTP id 8386540858; Thu, 22 Aug 2002

    07:35:02 -0400 (EDT)

Delivered-To: exmh-workers@listman.example.com

Received: from int-mx1.corp.example.com (int-mx1.corp.example.com

    [172.1

There is a tremendous amount of information in each email. To keep this project in scope, let's just use the subject line as our predictor. We need to use string commands from the `re` package to extract the subject line from each file. We will create a list of the subject line, the predictor, and the class of the email, spam or not. We determine class from the folder name. 

The command `re.sub` requires a regular expression. We will use `^Subject` as this tells the function we want to find  at the start of the string, https://docs.python.org/2/library/re.html. The method `strip` takes out the white spaces at the beginning and end of the string.

To read in the files we will use `latin1` codec because the default and standard `utf-8` produce an error. We could ignore the error as well. To see more information on this go to http://balusc.omnifaces.org/2009/05/unicode-how-to-get-characters-right.html

In [10]:
data=[]
for fn in glob.glob(path):
    is_spam = "ham" not in fn
    
    with open(fn,'r',encoding = 'latin1') as file:
        # could use errors='ignore' instead of coding
        for line in file:
            if line.startswith("Subject:"):
                subject=re.sub(r"^Subject: ","",line).strip()
                data.append((subject,is_spam))

Printing out the first 10 elements of the data list gives an idea of the emails' subject lines.

In [40]:
data[:10]

[('Re: New Sequences Window', False),
 ('[zzzzteana] RE: Alexander', False),
 ('[zzzzteana] Moscow bomber', False),
 ("[IRR] Klez: The Virus That  Won't Die", False),
 ('Re: Insert signature', False),
 ('Re: [zzzzteana] Nothing like mama used to make', False),
 ('Re: [zzzzteana] Nothing like mama used to make', False),
 ('[zzzzteana] Playboy wants to go out with a bang', False),
 ('Re: [zzzzteana] Nothing like mama used to make', False),
 ('[zzzzteana] Meaningful sentences', False)]

We are not sure what zzzzteana means and whether it is only associated with ham. This may be something we remove later.

In [41]:
len(data) # Number of emails

3423

In [38]:
sum(list(zip(*data))[1]) #Number of spam emails

503

### 4. Feature Engineering  

We are dealing with string data as our predictor. We first need to clean it up. The choices we make here will potentially have a big impact on the quality of the model. Ideally we would go back and test the sensitivity of our results to these choices.

First we will make all the text lower case. This will ensure that words such as Free and free are viewed as equivalent. This may not be a good idea for spam as an all capitals word might be more indicative of spam.

In [24]:
# Example of the function we need.
'FREE'.lower()

'free'

Next, we need to tokenize our string. This means to split the string into a list of words. This requires the use of regular expressions, https://docs.python.org/2/library/re.html. 

As an example, we will use the first ten subject lines of the data object.

In [26]:
for subject, is_spam in data[:10]:
    print(re.findall("[a-z0-9']+",subject.lower()))

['re', 'new', 'sequences', 'window']
['zzzzteana', 're', 'alexander']
['zzzzteana', 'moscow', 'bomber']
['irr', 'klez', 'the', 'virus', 'that', "won't", 'die']
['re', 'insert', 'signature']
['re', 'zzzzteana', 'nothing', 'like', 'mama', 'used', 'to', 'make']
['re', 'zzzzteana', 'nothing', 'like', 'mama', 'used', 'to', 'make']
['zzzzteana', 'playboy', 'wants', 'to', 'go', 'out', 'with', 'a', 'bang']
['re', 'zzzzteana', 'nothing', 'like', 'mama', 'used', 'to', 'make']
['zzzzteana', 'meaningful', 'sentences']


Finally, it is a common step in text preparation to remove common words, called stop words, for example the word the. These words are common and tend not to aid in prediction. 

An easy source for these words comes from the `nltk`, natural language toolkit, package.

In [25]:
import nltk
# Getting the English stop words from nltk
stop_words = nltk.corpus.stopwords.words('english')

# Printing out the first eight stop words
stop_words[:8]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves']

In [27]:
for subject, is_spam in data[:10]:
    print(re.findall("[a-z0-9']+",subject.lower()))
    words = re.findall("[a-z0-9']+",subject.lower())
    [words.remove(word) for word in words if word in stop_words]
    print(words)

['re', 'new', 'sequences', 'window']
['new', 'sequences', 'window']
['zzzzteana', 're', 'alexander']
['zzzzteana', 'alexander']
['zzzzteana', 'moscow', 'bomber']
['zzzzteana', 'moscow', 'bomber']
['irr', 'klez', 'the', 'virus', 'that', "won't", 'die']
['irr', 'klez', 'virus', "won't", 'die']
['re', 'insert', 'signature']
['insert', 'signature']
['re', 'zzzzteana', 'nothing', 'like', 'mama', 'used', 'to', 'make']
['zzzzteana', 'nothing', 'like', 'mama', 'used', 'make']
['re', 'zzzzteana', 'nothing', 'like', 'mama', 'used', 'to', 'make']
['zzzzteana', 'nothing', 'like', 'mama', 'used', 'make']
['zzzzteana', 'playboy', 'wants', 'to', 'go', 'out', 'with', 'a', 'bang']
['zzzzteana', 'playboy', 'wants', 'go', 'with', 'bang']
['re', 'zzzzteana', 'nothing', 'like', 'mama', 'used', 'to', 'make']
['zzzzteana', 'nothing', 'like', 'mama', 'used', 'make']
['zzzzteana', 'meaningful', 'sentences']
['zzzzteana', 'meaningful', 'sentences']


Before going further, let's summarize the data. We want to count the total frequency of words in both the spam and ham data sets. Then print those out.

First, let's write a function that will tokenize our data and remove stop words.

In [49]:
def token1(subject,sw):
    words = re.findall("[a-z0-9']+",subject.lower())
    [words.remove(word) for word in words if word in sw]
    return words

In [51]:
print(token1(data[1][0],stop_words))

['zzzzteana', 'alexander']


https://www.analyticsvidhya.com/blog/2017/09/naive-bayes-explained/
http://localhost:8888/notebooks/Documents/Classes/Books/Stats/Python%20Data%20Science%20Handbook/PythonDataScienceHandbook-master/notebooks/05.05-Naive-Bayes.ipynb

## In Depth: Naive Bayes Classification

The previous four sections have given a general overview of the concepts of machine learning.
In this section and the ones that follow, we will be taking a closer look at several specific algorithms for supervised and unsupervised learning, starting here with naive Bayes classification.

Naive Bayes models are a group of extremely fast and simple classification algorithms that are often suitable for very high-dimensional datasets.
Because they are so fast and have so few tunable parameters, they end up being very useful as a quick-and-dirty baseline for a classification problem.
This section will focus on an intuitive explanation of how naive Bayes classifiers work, followed by a couple examples of them in action on some datasets.

### Bayesian Classification

Naive Bayes classifiers are built on Bayesian classification methods.
These rely on Bayes's theorem, which is an equation describing the relationship of conditional probabilities of statistical quantities.
In Bayesian classification, we're interested in finding the probability of a label given some observed features, which we can write as $P(L~|~{\rm features})$.
Bayes's theorem tells us how to express this in terms of quantities we can compute more directly:

$$
P(L~|~{\rm features}) = \frac{P({\rm features}~|~L)P(L)}{P({\rm features})}
$$

If we are trying to decide between two labels—let's call them $L_1$ and $L_2$—then one way to make this decision is to compute the ratio of the posterior probabilities for each label:

$$
\frac{P(L_1~|~{\rm features})}{P(L_2~|~{\rm features})} = \frac{P({\rm features}~|~L_1)}{P({\rm features}~|~L_2)}\frac{P(L_1)}{P(L_2)}
$$

All we need now is some model by which we can compute $P({\rm features}~|~L_i)$ for each label.
Such a model is called a *generative model* because it specifies the hypothetical random process that generates the data.
Specifying this generative model for each label is the main piece of the training of such a Bayesian classifier.
The general version of such a training step is a very difficult task, but we can make it simpler through the use of some simplifying assumptions about the form of this model.

This is where the "naive" in "naive Bayes" comes in: if we make very naive assumptions about the generative model for each label, we can find a rough approximation of the generative model for each class, and then proceed with the Bayesian classification.
Different types of naive Bayes classifiers rest on different naive assumptions about the data, and we will examine a few of these in the following sections.

We begin with the standard imports:

## When to Use Naive Bayes

Because naive Bayesian classifiers make such stringent assumptions about data, they will generally not perform as well as a more complicated model.
That said, they have several advantages:

- They are extremely fast for both training and prediction
- They provide straightforward probabilistic prediction
- They are often very easily interpretable
- They have very few (if any) tunable parameters

These advantages mean a naive Bayesian classifier is often a good choice as an initial baseline classification.
If it performs suitably, then congratulations: you have a very fast, very interpretable classifier for your problem.
If it does not perform well, then you can begin exploring more sophisticated models, with some baseline knowledge of how well they should perform.

Naive Bayes classifiers tend to perform especially well in one of the following situations:

- When the naive assumptions actually match the data (very rare in practice)
- For very well-separated categories, when model complexity is less important
- For very high-dimensional data, when model complexity is less important

The last two points seem distinct, but they actually are related: as the dimension of a dataset grows, it is much less likely for any two points to be found close together (after all, they must be close in *every single dimension* to be close overall).
This means that clusters in high dimensions tend to be more separated, on average, than clusters in low dimensions, assuming the new dimensions actually add information.
For this reason, simplistic classifiers like naive Bayes tend to work as well or better than more complicated classifiers as the dimensionality grows: once you have enough data, even a simple model can be very powerful.